目录

一、分组模式及其对象

    1. 分组的一般模式
    2. 分组依据的本质
    练一练
    3. Groupby对象
    4. 分组的三大操作

二、聚合函数

    1. 内置聚合函数
    练一练
    2. agg方法
    练一练

三、变换和过滤

    1. 变换函数与transform方法
    练一练
    2. 组索引与过滤
    练一练

四、跨列分组

    1. apply的引入
    2. apply的使用
    练一练

五、练习



In [1]:
import numpy as np
import pandas as pd

# 一、分组模式及其对象

1、分组的一般模式

df.groupby(分组依据)[数据来源].使用操作

In [2]:
#按照性别统计身高中位数

df = pd.read_csv('./data/learn_pandas.csv')
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

2、分组依据的本质

In [3]:
#根据多个维度进行分组，在groupby中传入相应列名构成的列表即可。

#根据学校和性别进行分组，统计身高的均值
df.groupby(['School', 'Gender'])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

In [4]:
#根据学生体重是否超过总体均值来分组，计算身高的均值。

#首先应该先写出分组条件：
condition = df.Weight > df.Weight.mean()

#然后将其传入groupby中：
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

【练一练】

请根据上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值。

In [6]:
weight_level = df.Weight.quantile([0.25,0.75])
# print(type(weight_level))
# weight_level['0.25']
# weight_level[0.25]
temp1 = weight_level[0.25]
temp2 = weight_level[0.75]
df.groupby(df.Weight.apply(lambda x:'low' if x<temp1 else('normal' if x<temp2 else 'high'))).Height.mean()



Weight
high      172.920755
low       153.753659
normal    161.800000
Name: Height, dtype: float64

In [8]:
#最后产生的结果就是按照条件列表中元素的值（此处是True和False）来分组

#用随机传入字母序列
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()

a    162.613115
b    162.529310
c    164.418750
Name: Height, dtype: float64

In [9]:
#传入多个序列进入groupby，最后分组的依据就是这两个序列对应行的唯一组合：

df.groupby([condition, item])['Height'].mean()

Weight   
False   a    159.535556
        b    158.425641
        c    159.062791
True    a    171.268750
        b    170.952632
        c    175.385714
Name: Height, dtype: float64

In [10]:
#之前传入列名只是一种简便的记号，事实上等价于传入的是一个或多个列，最后分组的依据来自于数据来源组合的unique值，通过drop_duplicates就能知道具体的组类别：

df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,Shanghai Jiao Tong University,Female
1,Peking University,Male
2,Shanghai Jiao Tong University,Male
3,Fudan University,Female
4,Fudan University,Male
5,Tsinghua University,Female
9,Peking University,Female
16,Tsinghua University,Male


In [11]:
df.groupby([df['School'], df['Gender']])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

3、Groupby对象

In [12]:
gb = df.groupby(['School', 'Grade'])
gb

In [13]:
#通过ngroups属性，得到分组个数：

gb.ngroups

16

In [14]:
#通过groups属性，返回从 组名 映射到 组索引列表 的字典：

res = gb.groups
res.keys() # 字典的值由于是索引，元素个数过多，此处只展示字典的键

dict_keys([('Fudan University', 'Freshman'), ('Fudan University', 'Junior'), ('Fudan University', 'Senior'), ('Fudan University', 'Sophomore'), ('Peking University', 'Freshman'), ('Peking University', 'Junior'), ('Peking University', 'Senior'), ('Peking University', 'Sophomore'), ('Shanghai Jiao Tong University', 'Freshman'), ('Shanghai Jiao Tong University', 'Junior'), ('Shanghai Jiao Tong University', 'Senior'), ('Shanghai Jiao Tong University', 'Sophomore'), ('Tsinghua University', 'Freshman'), ('Tsinghua University', 'Junior'), ('Tsinghua University', 'Senior'), ('Tsinghua University', 'Sophomore')])

【练一练】
上一小节介绍了可以通过drop_duplicates得到具体的组类别，现请用groups属性完成类似的功能。

In [15]:
gb = df.groupby(['School', 'Grade'])
res = gb.groups
res.keys()

dict_keys([('Fudan University', 'Freshman'), ('Fudan University', 'Junior'), ('Fudan University', 'Senior'), ('Fudan University', 'Sophomore'), ('Peking University', 'Freshman'), ('Peking University', 'Junior'), ('Peking University', 'Senior'), ('Peking University', 'Sophomore'), ('Shanghai Jiao Tong University', 'Freshman'), ('Shanghai Jiao Tong University', 'Junior'), ('Shanghai Jiao Tong University', 'Senior'), ('Shanghai Jiao Tong University', 'Sophomore'), ('Tsinghua University', 'Freshman'), ('Tsinghua University', 'Junior'), ('Tsinghua University', 'Senior'), ('Tsinghua University', 'Sophomore')])

In [16]:
#当size作为DataFrame的属性时，返回的是表长乘以表宽的大小
#在groupby对象上表示统计每个组的元素个数：

gb.size()

School                         Grade    
Fudan University               Freshman      9
                               Junior       12
                               Senior       11
                               Sophomore     8
Peking University              Freshman     13
                               Junior        8
                               Senior        8
                               Sophomore     5
Shanghai Jiao Tong University  Freshman     13
                               Junior       17
                               Senior       22
                               Sophomore     5
Tsinghua University            Freshman     17
                               Junior       22
                               Senior       14
                               Sophomore    16
dtype: int64

In [17]:
#通过get_group方法可以直接获取所在组对应的行，此时必须知道组的具体名字：

gb.get_group(('Fudan University', 'Freshman'))

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
15,Fudan University,Freshman,Changqiang Yang,Female,156.0,49.0,N,3,2020/1/1,0:05:25
28,Fudan University,Freshman,Gaoqiang Qin,Female,170.2,63.0,N,2,2020/1/7,0:05:24
63,Fudan University,Freshman,Gaofeng Zhao,Female,152.2,43.0,N,2,2019/10/31,0:04:00
70,Fudan University,Freshman,Yanquan Wang,Female,163.5,55.0,N,1,2019/11/19,0:04:07
73,Fudan University,Freshman,Feng Wang,Male,176.3,74.0,N,1,2019/9/26,0:03:31
105,Fudan University,Freshman,Qiang Shi,Female,164.5,52.0,N,1,2019/12/11,0:04:23
108,Fudan University,Freshman,Yanqiang Xu,Female,152.4,38.0,N,1,2019/12/8,0:05:03
157,Fudan University,Freshman,Xiaoli Lv,Female,152.5,45.0,N,2,2019/9/11,0:04:17
186,Fudan University,Freshman,Yanjuan Zhao,Female,NaN,53.0,N,2,2019/10/9,0:04:21


4、分组的三大操作

例1：依据 `性别` 分组，统计全国人口 `寿命` 的 平均值
    每一个组返回一个标量值，可以是平均值、中位数、组容量`size`等

例2：依据 `季节` 分组，对每一个季节的 `温度` 进行 `组内标准化`
    做了原序列的标准化处理，也就是说每组返回的是一个`Series`类型

例3：依据 `班级` 分组，筛选出组内 `数学分数` 的 平均值超过80分的班级
    既不是标量也不是序列，返回的整个组所在行的本身，即返回了`DataFrame`类型
    
由此，引申出分组的三大操作：聚合、变换和过滤，分别对应了三个例子的操作，下面就要分别介绍相应的`agg`、`transform`和`filter`函数及其操作。

# 二、聚合函数

1、内置聚合函数

直接定义在groupby对象的聚合函数，因为它的速度基本都会经过内部的优化，使用功能时应当优先考虑。根据返回标量值的原则，包括如下函数：`max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod`。

In [18]:
gb = df.groupby('Gender')['Height']

gb.idxmin()

Gender
Female    143
Male      199
Name: Height, dtype: int64

In [19]:
#最大值去95%分位数
gb.quantile(0.95)

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

【练一练】
请查阅文档，明确all/any/mad/skew/sem/prod函数的含义。

In [20]:
#是否所有元素都为真
gb.all()

Gender
Female    True
Male      True
Name: Height, dtype: bool

In [21]:
#一个序列中满足一个True，则返回True
gb.any()

Gender
Female    True
Male      True
Name: Height, dtype: bool

In [22]:
#根据平均值计算平均绝对距离差
gb.mad()

Gender
Female    4.088108
Male      5.394617
Name: Height, dtype: float64

In [23]:
# 返回峰度
gb.skew()

Gender
Female   -0.219253
Male      0.437535
Name: Height, dtype: float64

In [24]:
# 返回所请求轴上的平均值的无偏标准误差
gb.sem()

Gender
Female    0.439893
Male      0.986985
Name: Height, dtype: float64

In [25]:
# 返回乘积
gb.prod()

Gender
Female    4.232080e+290
Male      1.594210e+114
Name: Height, dtype: float64

In [26]:
#当传入的数据来源包含多个列时，将按照列进行迭代计算：

gb = df.groupby('Gender')[['Height', 'Weight']]
gb.max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


2、agg方法

groupby对象上的函数的不便之处：
- 无法同时使用多个函数
- 无法对特定的列使用特定的聚合函数
- 无法使用自定义的聚合函数
- 无法直接对结果的列名在聚合前进行自定义命名

通过`agg`函数解决

【a】使用多个函数

In [27]:
#当使用多个聚合函数时，需要用列表的形式把内置聚合函数对应的字符串传入，先前提到的所有字符串都是合法的。

gb.agg(['sum', 'idxmax', 'skew'])

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

In [28]:
#【b】对特定的列使用特定的聚合函数

#通过构造字典传入agg中实现，其中字典以列名为键，以聚合字符串或字符串列表为值。

gb.agg({'Height':['mean','max'], 'Weight':'count'})

Height        Weight
             mean    max  count
Gender                         
Female  159.19697  170.2    135
Male    173.62549  193.9     54

【练一练】
请使用【b】中的传入字典的方法完成【a】中等价的聚合任务。

In [29]:
gb.agg({'Height':['sum', 'idxmax', 'skew'], 'Weight':['sum', 'idxmax', 'skew']})

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

In [30]:
#【c】使用自定义函数

#在agg中可以使用具体的自定义函数， 需要注意传入函数的参数是之前数据源中的列，逐列进行计算 。
#下面分组计算身高和体重的极差：

gb.agg(lambda x: x.mean()-x.min())

,Height,Weight
Gender,,
Female,13.79697,13.918519
Male,17.92549,21.759259


【练一练】
在groupby对象中可以使用describe方法进行统计信息汇总，请同时使用多个聚合函数，完成与该方法相同的功能。

In [31]:
gb.describe()

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

In [72]:
gb.agg(['count', 'mean', 'std', 'min', 'max'])

Height                                    Weight                       \
        count       mean       std    min    max  count       mean       std   
Gender                                                                         
Female    132  159.19697  5.053982  145.4  170.2    135  47.918519  5.405983   
Male       51  173.62549  7.048485  155.7  193.9     54  72.759259  7.772557   

                    
         min   max  
Gender              
Female  34.0  63.0  
Male    51.0  89.0

In [87]:
gh = df.groupby('Gender')['Height']
gh.describe()



,count,mean,std,min,25%,50%,75%,max
Gender,,,,,,,,
Female,132.0,159.19697,5.053982,145.4,155.675,159.6,162.825,170.2
Male,51.0,173.62549,7.048485,155.7,168.900,173.4,177.150,193.9


In [73]:
#由于传入的是序列，因此序列上的方法和属性都是可以在函数中使用的，只需保证返回值是标量即可。

#如果组的指标均值，超过该指标的总体均值，返回High，否则返回Low。
def my_func(s):
    res = 'High'
    if s.mean() <= df[s.name].mean():
        res = 'Low'
    return res
gb.agg(my_func)

,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


In [74]:
#【d】聚合结果重命名

#如果想要对聚合结果的列名进行重命名，只需要将上述函数的位置改写成元组，
#元组的第一个元素为新的名字，第二个位置为原来的函数，包括聚合字符串和自定义函数

gb.agg([('range', lambda x: x.max()-x.min()), ('my_sum', 'sum')])


Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

In [75]:
gb.agg({'Height': [('my_func', my_func), 'sum'], 'Weight': lambda x:x.max()})

Height            Weight
       my_func      sum <lambda>
Gender                          
Female     Low  21014.0     63.0
Male      High   8854.9     89.0

In [76]:
#使用对一个或者多个列使用单个聚合的时候，重命名需要加方括号，否则就不知道是新的名字还是手误输错的内置函数字符串：

gb.agg([('my_sum', 'sum')])

,Height,Weight
,my_sum,my_sum
Gender,,
Female,21014.0,6469.0
Male,8854.9,3929.0


In [79]:
gb.agg({'Height': [('my_func', my_func), 'sum'], 'Weight': [('range', lambda x:x.max())]})

Height          Weight
       my_func      sum  range
Gender                        
Female     Low  21014.0   63.0
Male      High   8854.9   89.0

# 三、变换和过滤

1、变换函数与transform方法

变换函数的返回值为同长度的序列，最常用的内置变换函数是累计函数：`cumcount/cumsum/cumprod/cummax/cummin`，它们的使用方式和聚合函数类似，只不过完成的是组内累计操作。此外在`groupby`对象上还定义了填充类和滑窗类的变换函数

In [80]:
#样本的累计最大值
gb.cummax().head()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0


【练一练】
在groupby对象中，rank方法也是一个实用的变换函数，请查阅它的功能并给出一个使用的例子。

In [82]:
gb.rank()

,Height,Weight
0,58.0,47.5
1,5.0,19.0
2,50.0,54.0
3,NaN,14.5
4,27.0,31.5
...,...,...
195,21.5,47.5
196,83.0,83.0
197,21.5,42.0
198,31.0,23.0


In [ ]:
#沿轴计算数值数据等级（1到n）。
#默认情况下，为相等的值分配一个等级，该等级是这些值的等级的平均值。

In [99]:
#0,6,0,9排序后是0，0，6，9，等级对应1，2，3，4，前面两个0一样，对应等级为（1+2）/2=1.5
test = pd.DataFrame(data = {'name':['Amy', 'John', 'Sim', 'Li'],
                           'Score':[0,6,0,9]})
test

,name,Score
0,Amy,0
1,John,6
2,Sim,0
3,Li,9


In [100]:
test['my_rank'] = test['Score'].rank()
test

,name,Score,my_rank
0,Amy,0,1.5
1,John,6,3.0
2,Sim,0,1.5
3,Li,9,4.0


In [102]:
#2,6,0,9排序后是0，2，6，9，等级对应1，2，3，4
test1 = pd.DataFrame(data = {'name':['Amy', 'John', 'Sim', 'Li'],
                           'Score':[2,6,0,9]})
test1

,name,Score
0,Amy,2
1,John,6
2,Sim,0
3,Li,9


In [104]:
test1['my_rank'] = test1['Score'].rank()
test1

,name,Score,my_rank
0,Amy,2,2.0
1,John,6,3.0
2,Sim,0,1.0
3,Li,9,4.0


In [105]:
#2,6,0,nan排序后是0，2，6，等级对应1，2，3
test2 = pd.DataFrame(data = {'name':['Amy', 'John', 'Sim', 'Li'],
                           'Score':[2,6,0,np.nan]})
test2

,name,Score
0,Amy,2.0
1,John,6.0
2,Sim,0.0
3,Li,NaN


In [106]:
test2['my_rank'] = test2['Score'].rank()
test2

,name,Score,my_rank
0,Amy,2.0,2.0
1,John,6.0,3.0
2,Sim,0.0,1.0
3,Li,NaN,NaN


In [107]:
#对身高和体重进行分组标准化，即减去组均值后除以组的标准差：

gb.transform(lambda x: (x-x.mean())/x.std()).head()

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631


【练一练】
对于transform方法无法像agg一样，通过传入字典来对指定列使用特定的变换，如果需要在一次transform的调用中实现这种功能，请给出解决方案。

In [108]:
#前面提到了transform只能返回同长度的序列，但事实上还可以返回一个标量，这会使得结果被广播到其所在的整个组，这种 标量广播 的技巧在特征工程中是非常常见的。例如，构造两列新特征来分别表示样本所在性别组的身高均值和体重均值：

gb.transform('mean').head() # 传入返回标量的函数也是可以的

,Height,Weight
0,159.19697,47.918519
1,173.62549,72.759259
2,173.62549,72.759259
3,159.19697,47.918519
4,173.62549,72.759259


2、组索引与过滤

索引和过滤有什么区别呢？

过滤在分组中是对于组的过滤，而索引是对于行的过滤，在第二章中的返回值，无论是布尔列表还是元素列表或者位置列表，本质上都是对于行的筛选，即如果符合筛选条件的则选入结果表，否则不选入。

组过滤作为行过滤的推广，指的是如果对一个组的全体所在行进行统计的结果返回`True`则会被保留，`False`则该组会被过滤，最后把所有未被过滤的组其对应的所在行拼接起来作为`DataFrame`返回。

在`groupby`对象中，定义了`filter`方法进行组的筛选，其中自定义函数的输入参数为数据源构成的`DataFrame`本身，在之前例子中定义的`groupby`对象中，传入的就是`df[['Height', 'Weight']]`，因此所有表方法和属性都可以在自定义函数中相应地使用，同时只需保证自定义函数的返回为布尔值即可。


In [109]:
#在原表中通过过滤得到所有容量大于100的组：

gb.filter(lambda x: x.shape[0] > 100).head()

,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0


【练一练】
从概念上说，索引功能是组过滤功能的子集，请使用filter函数完成loc[...]的功能，这里假设"..."是元素列表。

# 四、跨列分组

1、apply的引入

之前几节介绍了三大分组操作，但事实上还有一种常见的分组场景，无法用前面介绍的任何一种方法处理，例如现在如下定义身体质量指数BMI：
$${\rm BMI} = {\rm\frac{Weight}{Height^2}}$$
其中体重和身高的单位分别为千克和米，需要分组计算组BMI的均值。

首先，这显然不是过滤操作，因此`filter`不符合要求；其次，返回的均值是标量而不是序列，因此`transform`不符合要求；最后，似乎使用`agg`函数能够处理，但是之前强调过聚合函数是逐列处理的，而不能够$\color{#FF0000}{多列数据同时处理}$。由此，引出了`apply`函数来解决这一问题。

2、apply的应用

在设计上，`apply`的自定义函数传入参数与`filter`完全一致，只不过后者只允许返回布尔值。现如下解决上述计算问题：

In [122]:
def BMI(x):
    Height = x['Height']/100
    Weight = x['Weight']
    BMI_value = Weight/Height**2
    return BMI_value.mean()
gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

In [121]:
gb = df.groupby('Gender')[['Height', 'Weight']]

除了返回标量之外，apply方法还可以返回一维Series和二维DataFrame，但它们产生的数据框维数和多级索引的层数应当如何变化？

In [131]:
#【a】标量情况：结果得到的是 Series ，索引与 agg 的结果一致

gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]
gb.apply(lambda x: 0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
dtype: int64

In [132]:
gb.apply(lambda x: [0, 0]) # 虽然是列表，但是作为返回值仍然看作标量

Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

In [133]:
#【b】Series情况：得到的是DataFrame，行索引与标量情况一致，列索引为Series的索引

gb.apply(lambda x: pd.Series([0,0],index=['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

【练一练】
请尝试在apply传入的自定义函数中，根据组的某些特征返回相同长度但索引不同的Series，会报错吗？

In [134]:
#【c】DataFrame情况：得到的是DataFrame，
#行索引最内层在每个组原先agg的结果索引上，再加一层返回的DataFrame行索引，
#同时分组结果DataFrame的列索引和返回的DataFrame列索引一致。

gb.apply(lambda x: pd.DataFrame(np.ones((2,2)), index = ['a','b'], columns=pd.Index([('w','x'),('y','z')])))

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0
Male   1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0

【练一练】
请尝试在apply传入的自定义函数中，根据组的某些特征返回相同大小但列索引不同的DataFrame，会报错吗？如果只是行索引不同，会报错吗？

最后需要强调的是，apply函数的灵活性是以牺牲一定性能为代价换得的，除非需要使用跨列处理的分组处理，否则应当使用其他专门设计的groupby对象方法，否则在性能上会存在较大的差距。同时，在使用聚合函数和变换函数时，也应当优先使用内置函数，它们经过了高度的性能优化，一般而言在速度上都会快于用自定义函数来实现。

【练一练】
在groupby对象中还定义了cov和corr函数，从概念上说也属于跨列的分组处理。请利用之前定义的gb对象，使用apply函数实现与gb.cov()同样的功能并比较它们的性能。

In [137]:
gb.apply(lambda x: x.cov())

Height     Weight
Gender Test_Number                             
Female 1           Height  20.963600  21.452034
                   Weight  21.452034  26.438244
       2           Height  31.615680  30.386170
                   Weight  30.386170  34.568250
       3           Height  23.582395  20.801307
                   Weight  20.801307  23.228070
Male   1           Height  42.638234  48.785833
                   Weight  48.785833  67.669951
       2           Height  57.041732  38.224183
                   Weight  38.224183  37.869281
       3           Height  56.157667  84.020000
                   Weight  84.020000  89.904762

# 五、练习
### Ex1：汽车数据集
现有一份汽车数据集，其中`Brand, Disp., HP`分别代表汽车品牌、发动机蓄量、发动机输出。

In [139]:
df = pd.read_csv('./data/car.csv')
df.head(3)

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63


In [140]:
#1. 先过滤出所属`Country`数超过2个的汽车，即若该汽车的`Country`在总体数据集中出现次数不超过2则剔除，
#再按`Country`分组计算价格均值、价格变异系数、该`Country`的汽车数量，其中变异系数的计算方法是标准差除以均值，
#并在结果中把变异系数重命名为`CoV`。

df.groupby('Country').filter(lambda x:x.shape[0]>2).groupby(
           'Country')['Price'].agg([(
           'CoV', lambda x: x.std()/x.mean()), 'mean', 'count'])



,CoV,mean,count
Country,,,
Japan,0.387429,13938.052632,19
Japan/USA,0.240040,10067.571429,7
Korea,0.243435,7857.333333,3
USA,0.203344,12543.269231,26


In [141]:
#2. 按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计`Price`的均值。
df.shape[0]

60

In [142]:
condition = ['Head']*20+['Mid']*20+['Tail']*20

df.groupby(condition)['Price'].mean()

Head     9069.95
Mid     13356.40
Tail    15420.65
Name: Price, dtype: float64

In [143]:
#3. 对类型`Type`分组，对`Price`和`HP`分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单层索引。
res = df.groupby('Type').agg({'Price': ['max'], 'HP': ['min']})

res.columns = res.columns.map(lambda x:'_'.join(x))

res

,Price_max,HP_min
Type,,
Compact,18900,95
Large,17257,150
Medium,24760,110
Small,9995,63
Sporty,13945,92
Van,15395,106


In [144]:
#4. 对类型`Type`分组，对`HP`进行组内的`min-max`归一化。
def normalize(s):
    s_min, s_max = s.min(), s.max()
    res = (s - s_min)/(s_max - s_min)
    return res


df.groupby('Type')['HP'].transform(normalize).head()

0    1.00
1    0.54
2    0.00
3    0.58
4    0.80
Name: HP, dtype: float64

In [145]:
#5. 对类型`Type`分组，计算`Disp.`与`HP`的相关系数。
df.groupby('Type')[['HP', 'Disp.']].apply(
   lambda x:np.corrcoef(x['HP'].values, x['Disp.'].values)[0,1])

Type
Compact    0.586087
Large     -0.242765
Medium     0.370491
Small      0.603916
Sporty     0.871426
Van        0.819881
dtype: float64

### Ex2：实现transform函数
* `groupby`对象的构造方法是`my_groupby(df, group_cols)`
* 支持单列分组与多列分组
* 支持带有标量广播的`my_groupby(df)[col].transform(my_func)`功能
* `pandas`的`transform`不能跨列计算，请支持此功能，即仍返回`Series`但`col`参数为多列
* 无需考虑性能与异常处理，只需实现上述功能，在给出测试样例的同时与`pandas`中的`transform`对比结果是否一致

In [146]:
class my_groupby:
    def __init__(self, my_df, group_cols):
        self.my_df = my_df.copy()
        self.groups = my_df[group_cols].drop_duplicates()
        if isinstance(self.groups, pd.Series):
            self.groups = self.groups.to_frame()
        self.group_cols = self.groups.columns.tolist()
        self.groups = {i: self.groups[i].values.tolist(
                       ) for i in self.groups.columns}
        self.transform_col = None
    def __getitem__(self, col):
        self.pr_col = [col] if isinstance(col, str) else list(col)
        return self
    def transform(self, my_func):
        self.num = len(self.groups[self.group_cols[0]])
        L_order, L_value = np.array([]), np.array([])
        for i in range(self.num):
            group_df = self.my_df.reset_index().copy()
            for col in self.group_cols:
                group_df = group_df[group_df[col]==self.groups[col][i]]
            group_df = group_df[self.pr_col]
            if group_df.shape[1] == 1:
                group_df = group_df.iloc[:, 0]
            group_res = my_func(group_df)
            if not isinstance(group_res, pd.Series):
                group_res = pd.Series(group_res,
                                      index=group_df.index,
                                      name=group_df.name)
            L_order = np.r_[L_order, group_res.index]
            L_value = np.r_[L_value, group_res.values]
        self.res = pd.Series(pd.Series(L_value, index=L_order).sort_index(
                   ).values,index=self.my_df.reset_index(
                   ).index, name=my_func.__name__)
        return self.res


my_groupby(df, 'Type')

In [147]:
#单列分组：

def f(s):
    res = (s-s.min())/(s.max()-s.min())
    return res


my_groupby(df, 'Type')['Price'].transform(f).head()

0    0.733592
1    0.372003
2    0.109712
3    0.186244
4    0.177525
Name: f, dtype: float64

In [148]:
df.groupby('Type')['Price'].transform(f).head()


0    0.733592
1    0.372003
2    0.109712
3    0.186244
4    0.177525
Name: Price, dtype: float64

In [149]:
#多列分组：

my_groupby(df, ['Type','Country'])['Price'].transform(f)

0     1.000000
1     0.000000
2     0.000000
3     0.000000
4     0.196357
5          NaN
6     0.361571
7     1.000000
8     1.000000
9     0.000000
10    1.000000
11    0.166622
12         NaN
13    0.744109
14    0.000000
15    1.000000
16    0.713105
17    0.000000
18    1.000000
19    0.846527
20    0.458867
21    0.807277
22         NaN
23    0.740082
24    0.572503
25    1.000000
26    0.000000
27    0.672917
28    1.000000
29    0.000000
30    1.000000
31    0.095936
32    0.350205
33         NaN
34    0.000000
35    0.092708
36         NaN
37    1.000000
38    0.170263
39    0.000000
40    1.000000
41    0.742137
42    0.181960
43    0.645958
44         NaN
45    0.787203
46    0.000000
47    0.170263
48    0.519886
49    0.524559
50    0.592972
51    0.000000
52    1.000000
53    0.552430
54    1.000000
55    0.000000
56    1.000000
57    0.984925
58    0.000000
59    0.854271
Name: f, dtype: float64

In [150]:
df.groupby(['Type','Country'])['Price'].transform(f).head()

0    1.000000
1    0.000000
2    0.000000
3    0.000000
4    0.196357
Name: Price, dtype: float64

In [151]:
#标量广播：

my_groupby(df, 'Type')['Price'].transform(lambda x:x.mean()).head()

0    7682.384615
1    7682.384615
2    7682.384615
3    7682.384615
4    7682.384615
Name: <lambda>, dtype: float64

In [152]:
df.groupby('Type')['Price'].transform(lambda x:x.mean()).head()

0    7682.384615
1    7682.384615
2    7682.384615
3    7682.384615
4    7682.384615
Name: Price, dtype: float64

In [153]:
#跨列计算：

my_groupby(df, 'Type')['Disp.', 'HP'].transform(
               lambda x: x['Disp.']/x.HP).head()

0    0.858407
1    1.266667
2    1.285714
3    0.989130
4    1.097087
Name: <lambda>, dtype: float64